In [1]:
# Step 2: Import pandas
import pandas as pd
from IPython.display import display

In [6]:
# Step 3: Load the Excel file
df = pd.read_excel("Book11.xlsx")

# Step 4: Display the DataFrame
df.head()

,Car Number,SOC Initial,Preference Slots,Route,Priority,Battery Capacity,Full Charge Range,KM before CS,Required time(If user provide),SOC Final,Required Time (Calculated)
0,23123,35,1,1,0,90.0,350.0,25.0,15.0,30,16
1,657,6,1,3,0,NaN,NaN,NaN,13.0,3,12
2,45345,28,1,1,1,NaN,NaN,NaN,15.0,22,13
3,768678,44,1,1,2,NaN,NaN,NaN,NaN,40,14
4,6756,12,1,1,3,NaN,NaN,NaN,NaN,8,15


In [7]:
# Step 3: Load the Excel file
df2 = pd.read_excel("CS2_freetime.xlsx")
cs2_Free_Slot_duration = df2['Vacancy Duration'].tolist()
# Create a new DataFrame to store deleted rows
CS2_vehicles = pd.DataFrame()
ignored_vehicles = pd.DataFrame()


In [8]:
Preference_Slots = df['Preference Slots'].unique().tolist()
Preference_Slots.sort()
print(Preference_Slots)

[1, 2, 3]


Route 1 : The path which returns back to the city.\
Route 2 : The path which leads to the outside of the city.\
Route 3 : The path which leads to the charging station 2. 

In [9]:
for i,j in zip(Preference_Slots,cs2_Free_Slot_duration):
    # Select the rows where 'Preference Slots' is 1
    df_preference_1 = df[df['Preference Slots'] == i]
    # Drop rows where 'SOC Final' is below 10
    df_preference_1_filtered = df_preference_1[df_preference_1['SOC Final'] >= 10]
    # Reorder the rows according to the values of 'priority' & then 'SOC Final' in descending order
    df_preference_1_filtered_sorted = df_preference_1_filtered.sort_values(by=['Priority','Route','SOC Final'])
    display(df_preference_1_filtered_sorted)
    # Calculate the sum of the 'Required Time (Calculated)' column
    total_required_time = df_preference_1_filtered_sorted['Required Time (Calculated)'].sum()

    # Initialize X with the calculated sum
    X = total_required_time

    # Traverse from below to top and delete rows where 'Route' is 2 until X < 180
    data_reversed = df_preference_1_filtered_sorted.iloc[::-1]
    if X > 180 :
        for index, row in data_reversed.iterrows():
            if row['Route'] == 3 and row['SOC Final'] >=30 :
                X -= row['Required Time (Calculated)']
                CS2_vehicles = CS2_vehicles.append(row)
                df_preference_1_filtered_sorted.drop(index, inplace=True)
                if X < 180:
                    break          
    if X > 180 :
        for index, row in data_reversed.iterrows():
            if row['Route'] == 1 :
                X -= row['Required Time (Calculated)']
                ignored_vehicles = ignored_vehicles.append(row)
                df_preference_1_filtered_sorted.drop(index, inplace=True)
                if X < 180:
                    break
    # Display the updated DataFrame and the final value of X
    updated_data = df_preference_1_filtered_sorted.reset_index(drop=True)
    print("Scheduled vehicles in slot "+ str(i))
    display(updated_data)
    print('Total time of charging in slot ' + str(i) + ' after scheduling the vehicle in CS1 is: '+ str(X))
    print("ignored vehicles of route 1 in slot "+ str(i))
    display(ignored_vehicles)
    print("vehicles left for CS2 in slot "+ str(i))
    display(CS2_vehicles)
    CS2_vehicles = CS2_vehicles.reset_index(drop=True)
    # Calculate the sum of the 'Required Time (Calculated)' column for charging station 2
    Y = CS2_vehicles['Required Time (Calculated)'].sum()
    print('total required time of vehicle of CS2 before scheduling: '+str(Y))
    if Y >=j :
        while Y >= j:
            min_index = CS2_vehicles['SOC Final'].idxmax()
            Y -= CS2_vehicles.loc[min_index, 'Required Time (Calculated)']
            CS2_vehicles.drop(min_index, inplace=True)
    print('total free time of CS2 for slot '+str(i)+'is: '+str(j))        
    print('total required time of vehicle of CS2 after scheduling: '+str(Y))
    print("Scheduled allocated vehicles of CS2 in slot "+ str(i))
    display(CS2_vehicles)
    # Drop all rows
    CS2_vehicles = CS2_vehicles.iloc[0:0]

    # Export each DataFrame to a separate CSV file
    file_name = f'slot_{i}_updated_cs1.csv'
    updated_data.to_csv(file_name, index=False)
    print(f'DataFrame {i+1} exported to {file_name}')

    # Export each DataFrame to a separate CSV file
    file_name = f'slot_{i}_ignored_cs1.csv'
    ignored_vehicles.to_csv(file_name, index=False)
    print(f'DataFrame {i+1} exported to {file_name}')

    ignored_vehicles = ignored_vehicles.iloc[0:0]

,Car Number,SOC Initial,Preference Slots,Route,Priority,Battery Capacity,Full Charge Range,KM before CS,Required time(If user provide),SOC Final,Required Time (Calculated)
0,23123,35,1,1,0,90.0,350.0,25.0,15.0,30,16
11,24,35,1,1,0,NaN,NaN,NaN,17.0,31,15
2,45345,28,1,1,1,NaN,NaN,NaN,15.0,22,13
15,76786,65,1,1,1,NaN,NaN,NaN,NaN,60,18
3,768678,44,1,1,2,NaN,NaN,NaN,NaN,40,14
13,5677,28,1,2,2,NaN,NaN,NaN,15.0,24,17
14,76767,21,1,1,3,NaN,NaN,NaN,NaN,17,20
16,678678,84,1,1,3,NaN,NaN,NaN,16.0,80,17
6,75675,14,1,2,3,NaN,NaN,NaN,NaN,10,16
8,79879,41,1,2,3,NaN,NaN,NaN,NaN,35,15


Scheduled vehicles in slot 1


C:\Users\USER\AppData\Local\Temp\ipykernel_22836\2815142082.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  CS2_vehicles = CS2_vehicles.append(row)
C:\Users\USER\AppData\Local\Temp\ipykernel_22836\2815142082.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  CS2_vehicles = CS2_vehicles.append(row)
C:\Users\USER\AppData\Local\Temp\ipykernel_22836\2815142082.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  CS2_vehicles = CS2_vehicles.append(row)
C:\Users\USER\AppData\Local\Temp\ipykernel_22836\2815142082.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ignored_vehicles = ignored_vehicles.append(row)
C:\Users\USER\AppData\Lo

,Car Number,SOC Initial,Preference Slots,Route,Priority,Battery Capacity,Full Charge Range,KM before CS,Required time(If user provide),SOC Final,Required Time (Calculated)
0,23123,35,1,1,0,90.0,350.0,25.0,15.0,30,16
1,24,35,1,1,0,NaN,NaN,NaN,17.0,31,15
2,45345,28,1,1,1,NaN,NaN,NaN,15.0,22,13
3,76786,65,1,1,1,NaN,NaN,NaN,NaN,60,18
4,768678,44,1,1,2,NaN,NaN,NaN,NaN,40,14
5,5677,28,1,2,2,NaN,NaN,NaN,15.0,24,17
6,75675,14,1,2,3,NaN,NaN,NaN,NaN,10,16
7,79879,41,1,2,3,NaN,NaN,NaN,NaN,35,15
8,7867,46,1,2,3,NaN,NaN,NaN,16.0,40,17
9,678678,62,1,2,3,NaN,NaN,NaN,NaN,56,15


Total time of charging in slot 1 after scheduling the vehicle in CS1 is: 170.0
ignored vehicles of route 1 in slot 1


...

vehicles left for CS2 in slot 3


,Car Number,SOC Initial,Preference Slots,Route,Priority,Battery Capacity,Full Charge Range,KM before CS,Required time(If user provide),SOC Final,Required Time (Calculated)


total required time of vehicle of CS2 before scheduling: 0.0
total free time of CS2 for slot 3is: 28
total required time of vehicle of CS2 after scheduling: 0.0
Scheduled allocated vehicles of CS2 in slot 3


,Car Number,SOC Initial,Preference Slots,Route,Priority,Battery Capacity,Full Charge Range,KM before CS,Required time(If user provide),SOC Final,Required Time (Calculated)


DataFrame 4 exported to slot_3_updated_cs1.csv
DataFrame 4 exported to slot_3_ignored_cs1.csv


If the next adjacent slot has more than 30 minutes free , the below algorthm will allocate the ignored vehicles of lowest priority route 1 from existing slot to the adjacent next slot. 

In [11]:
for i in Preference_Slots :
    if i != 3 :
        updated_list_of_next_slot = pd.read_csv("slot_"+str(i+1)+"_updated_cs1.csv")
        reversed_updated_list_of_next_slot = updated_list_of_next_slot.iloc[::-1].reset_index(drop=True)
        ignored_list = pd.read_csv("slot_"+str(i)+"_ignored_cs1.csv")
        Y = updated_list_of_next_slot['Required Time (Calculated)'].sum()
        if Y <= 150 and not ignored_list.empty:
            for index, row in ignored_list.iterrows():
                Y += row['Required Time (Calculated)']
                if Y > 180:
                    break
                reversed_updated_list_of_next_slot = reversed_updated_list_of_next_slot.append(row)
                ignored_list.drop(index, inplace=True)
        updated_list_of_next_slot = reversed_updated_list_of_next_slot.iloc[::-1].reset_index(drop=True)        
        print("updated list of slot "+str(i+1))        
        display(updated_list_of_next_slot)            

updated list of slot 2


,Car Number,SOC Initial,Preference Slots,Route,Priority,Battery Capacity,Full Charge Range,KM before CS,Required time(If user provide),SOC Final,Required Time (Calculated)
0,231233,35,2,1,0,90.0,350.0,25.0,15.0,30,16
1,240900,35,2,1,0,NaN,NaN,NaN,17.0,31,15
2,45397,28,2,1,1,NaN,NaN,NaN,15.0,22,13
3,765643,65,2,1,1,NaN,NaN,NaN,NaN,60,12
4,768666,44,2,1,2,NaN,NaN,NaN,NaN,40,14
5,564343,28,2,2,2,NaN,NaN,NaN,15.0,24,17
6,75699,14,2,2,3,NaN,NaN,NaN,NaN,10,16
7,798121,41,2,2,3,NaN,NaN,NaN,NaN,35,20
8,7867343,46,2,2,3,NaN,NaN,NaN,16.0,40,17
9,67862323,62,2,2,3,NaN,NaN,NaN,NaN,56,15


updated list of slot 3


C:\Users\USER\AppData\Local\Temp\ipykernel_22836\2166014108.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  reversed_updated_list_of_next_slot = reversed_updated_list_of_next_slot.append(row)


,Car Number,SOC Initial,Preference Slots,Route,Priority,Battery Capacity,Full Charge Range,KM before CS,Required time(If user provide),SOC Final,Required Time (Calculated)
0,671212.0,84.0,2.0,1.0,3.0,NaN,NaN,NaN,16.0,80.0,17.0
1,2312334.0,35.0,3.0,1.0,0.0,90.0,350.0,25.0,15.0,30.0,16.0
2,1110900.0,35.0,3.0,1.0,0.0,NaN,NaN,NaN,17.0,31.0,15.0
3,4539121.0,28.0,3.0,1.0,1.0,NaN,NaN,NaN,15.0,22.0,13.0
4,1118666.0,44.0,3.0,1.0,2.0,NaN,NaN,NaN,NaN,40.0,14.0
5,7569922.0,14.0,3.0,2.0,3.0,NaN,NaN,NaN,NaN,10.0,16.0
6,7981211.0,41.0,3.0,2.0,3.0,NaN,NaN,NaN,NaN,35.0,15.0
7,78673432.0,46.0,3.0,2.0,3.0,NaN,NaN,NaN,16.0,40.0,17.0
8,5645099.0,18.0,3.0,3.0,3.0,NaN,NaN,NaN,NaN,15.0,14.0
9,5645511.0,36.0,3.0,3.0,3.0,NaN,NaN,NaN,16.0,32.0,17.0
